### AlexNet (ImageNet) Architecture

* Original data : https://www.image-net.org/download.php
--------
* Subsample data : https://www.kaggle.com/competitions/imagenet-object-localization-challenge/data

#### Load packages

In [ ]:
import keras

In [ ]:
pip install tensorflow-datasets

In [ ]:
import pandas as pd

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
import tensorflow as tf 

In [ ]:
from matplotlib import pyplot as plt 

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
# Load the ImageNet2012 subset
# dataset, info = tfds.load('imagenet', with_info=True)

In [ ]:
def preprocess_data(image, label):
    image = tf.image.resize(image, (227, 227))  # Resize images to 227x227
    image = tf.cast(image, tf.float32) / 255.0  # Normalize images to [0, 1]
    return image, label

# Load CIFAR-10 dataset
(ds_train, ds_test), ds_info = tfds.load(
    'cifar10', 
    split=['train', 'test'], 
    shuffle_files=True, 
    as_supervised=True, 
    with_info=True
)

# Preprocess the datasets
BATCH_SIZE = 64
ds_train = ds_train.map(preprocess_data).shuffle(1024).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(preprocess_data).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
def AlexNet(input_shape=(227, 227, 3), classes=1000):
    model = Sequential()

    # 1st Convolutional Layer
    model.add(Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # 2nd Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(5, 5), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # 3rd Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3, 3), padding='same', activation='relu'))

    # 4th Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3, 3), padding='same', activation='relu'))

    # 5th Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # Passing it to a Fully Connected layer
    model.add(Flatten())
    # 1st Fully Connected Layer
    model.add(Dense(4096, activation='relu'))
    # Add Dropout to prevent overfitting
    model.add(Dropout(0.5))

    # 2nd Fully Connected Layer
    model.add(Dense(4096, activation='relu'))
    # Add Dropout
    model.add(Dropout(0.5))

    # 3rd Fully Connected Layer
    model.add(Dense(1000, activation='relu'))
    # Add Dropout
    model.add(Dropout(0.5))

    # Output Layer
    model.add(Dense(classes, activation='softmax'))

    return model

In [ ]:
# Create the model
alexnet_model = AlexNet()

# Compile the model
alexnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
alexnet_model.summary()


In [ ]:
run_hist = alexnet_model.fit(ds_train,
              epochs=5,
              validation_data=ds_test,
              shuffle=True )

In [ ]:
test_loss, test_acc = alexnet_model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')

In [ ]:
fig, ax = plt.subplots()
ax.plot(run_hist.history["loss"],'r', marker='.', label="Train Loss")
ax.plot(run_hist.history["val_loss"],'b', marker='.', label="Validation Loss")
ax.legend()